In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from datetime import datetime

Proposed Datasets
Show Info:
Season
Episode
Air date
Contestant 1
Contestant 2
Contestant 3
Returning Contestant

Show Details:
Episode
Round
Category
Clue
Value
Clue Order
Correct Response
Correct Responder
Incorrect Response 1
Incorrect Responder 1
Incorrect Response 2
Incorrect Responder 2
Incorrect Response 3
Incorrect Responder 3
Daily Double
Wager

Show Statistics:
J Round Score C1
J Round Score C2
J Round Score C3
DJ Round Score C1
DJ Round Score C2
DJ Round Score C3
FJ Wager C1
FJ Wager C2
FJ Wager C3
FJ Round Score C1
FJ Round Score C2
FJ Round Score C3


All Jeopardy! seasons:
http://www.j-archive.com/listseasons.php

## Season List

In [2]:
response = requests.get('http://www.j-archive.com/listseasons.php')

In [249]:
soup = BeautifulSoup(response.text)
#soup

In [4]:
season_urls = np.array(['http://www.j-archive.com/' + a['href'] for a in soup.findAll('a') 
                    if 'showseason' in a['href'] and 'http' not in a['href']])
season_urls

array(['http://www.j-archive.com/showseason.php?season=37',
       'http://www.j-archive.com/showseason.php?season=goattournament',
       'http://www.j-archive.com/showseason.php?season=36',
       'http://www.j-archive.com/showseason.php?season=35',
       'http://www.j-archive.com/showseason.php?season=34',
       'http://www.j-archive.com/showseason.php?season=33',
       'http://www.j-archive.com/showseason.php?season=32',
       'http://www.j-archive.com/showseason.php?season=31',
       'http://www.j-archive.com/showseason.php?season=30',
       'http://www.j-archive.com/showseason.php?season=29',
       'http://www.j-archive.com/showseason.php?season=28',
       'http://www.j-archive.com/showseason.php?season=27',
       'http://www.j-archive.com/showseason.php?season=26',
       'http://www.j-archive.com/showseason.php?season=25',
       'http://www.j-archive.com/showseason.php?season=24',
       'http://www.j-archive.com/showseason.php?season=23',
       'http://www.j-archive

In [5]:
seasons = pd.DataFrame(season_urls)
seasons.columns = ['season_url']
#seasons
seasons.to_csv('data/j_seasons.csv', index=False)

## Episode List

In [6]:
response = requests.get('http://www.j-archive.com/showseason.php?season=37')

In [250]:
soup = BeautifulSoup(response.text)
#soup

In [8]:
episodes = np.array([a['href'] for a in soup.findAll('a') if 'game_id' in a['href']])
episodes = pd.DataFrame(episodes)
episodes.columns = ['episode_url']

In [9]:
episodes.to_csv('data/j_episodes.csv', index=False)

## Episode Details

In [10]:
response = requests.get('http://www.j-archive.com/showgame.php?game_id=6895')

In [251]:
soup = BeautifulSoup(response.text)
#soup

In [13]:
header = soup.find('h1')
header = header.text
show_num = header.split('-')[0].strip().split('#')[1]
air_date = datetime.strftime(datetime.strptime(header.split('-')[1].strip().split(',',1)[1].strip(),'%B %d, %Y'), '%Y-%m-%d')
#print(show_num, ', ', air_date)

In [14]:
season = soup.find('a', text='[current season]')
print(season)
season = season['href'].split('=')[1]

<a href="http://www.j-archive.com/showseason.php?season=37">[current season]</a>


In [15]:
scores = soup.findAll(['h3','table'])
score_texts = np.array([['Scores at the end of the Jeopardy! Round:', 'J'],
                        ['Scores at the end of the Double Jeopardy! Round:', 'DJ'],
                        ['Final scores:', 'FJ']])
j_round = []
j_name = []
j_score = []
for x in score_texts:
    for element in scores:
        if element.text == x[0]:
            while element.name != 'table':
                element = element.next_sibling
                #print('name: ',element.name)
            score_table = element.findAll('tr') #
            [j_round.append(x[1]) for td in score_table[0].findAll('td')]
            [j_name.append(td.text) for td in score_table[0].findAll('td')]
            [j_score.append(td.text) for td in score_table[1].findAll('td')]

j_scores = pd.DataFrame({'show_num': show_num, 'round_code': j_round, 'contestant': j_name, 'score': j_score})
j_scores

,show_num,round_code,contestant,score
0,8305,J,Brayden,"$7,600"
1,8305,J,Amanda,"$1,600"
2,8305,J,Devon,"$2,400"
3,8305,DJ,Brayden,"$16,200"
4,8305,DJ,Amanda,"$14,200"
5,8305,DJ,Devon,"$1,200"
6,8305,FJ,Brayden,"$3,999"
7,8305,FJ,Amanda,"$3,200"
8,8305,FJ,Devon,"$1,200"


In [16]:
remarks_text = 'Coryat scores:'
j_right = []
j_ddright = []
j_wrong = []
j_ddwrong = []
for element in scores:
    if element.text == remarks_text:
        while element.name != 'table':
            element = element.next_sibling
        remarks_table = element.findAll('td', class_ = 'score_remarks')
        [j_right.append((re.findall('(\d{1,2}) R', x.text.split(',')[0]) or list('0'))[0]) for x in remarks_table]
        [j_ddright.append((re.findall('(\d{1}) DD', x.text.split(',')[0]) or list('0'))[0]) for x in remarks_table]
        [j_wrong.append((re.findall('(\d{1,2}) W', x.text.split(',')[1]) or list('0'))[0]) for x in remarks_table]
        [j_ddwrong.append((re.findall('(\d{1}) DD', x.text.split(',')[1]) or list('0'))[0]) for x in remarks_table]

j_remarks = pd.DataFrame({'contestant': j_name[:3], 'num_right': j_right, 'num_ddright': j_ddright, 'num_wrong': j_wrong,
                         'num_ddwrong': j_ddwrong})
j_remarks

,contestant,num_right,num_ddright,num_wrong,num_ddwrong
0,Brayden,26,1,4,1
1,Amanda,15,1,1,0
2,Devon,9,0,3,0


In [17]:
j_scores = j_scores.merge(j_remarks, how = 'left', on='contestant')
j_scores

,show_num,round_code,contestant,score,num_right,num_ddright,num_wrong,num_ddwrong
0,8305,J,Brayden,"$7,600",26,1,4,1
1,8305,J,Amanda,"$1,600",15,1,1,0
2,8305,J,Devon,"$2,400",9,0,3,0
3,8305,DJ,Brayden,"$16,200",26,1,4,1
4,8305,DJ,Amanda,"$14,200",15,1,1,0
5,8305,DJ,Devon,"$1,200",9,0,3,0
6,8305,FJ,Brayden,"$3,999",26,1,4,1
7,8305,FJ,Amanda,"$3,200",15,1,1,0
8,8305,FJ,Devon,"$1,200",9,0,3,0


In [18]:
finish_order = []
winner = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[0]
second = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[1]
third = j_scores[j_scores.round_code=='FJ'].sort_values(['score'], ascending = False)['contestant'].iloc[2]
finish_order.append([winner, 'Winner'])
finish_order.append([second, 'Second'])
finish_order.append([third, 'Third'])
finish_order = pd.DataFrame(finish_order)
#print(finish_order)
#print(type(finish_order))
#finish_order
finish_order.columns=(['contestant', 'finish'])
j_scores = j_scores.merge(finish_order, how = 'left', on = 'contestant')
j_scores
#!!!!add FJ wagers here!!!=========================================================================

,show_num,round_code,contestant,score,num_right,num_ddright,num_wrong,num_ddwrong,finish
0,8305,J,Brayden,"$7,600",26,1,4,1,Winner
1,8305,J,Amanda,"$1,600",15,1,1,0,Second
2,8305,J,Devon,"$2,400",9,0,3,0,Third
3,8305,DJ,Brayden,"$16,200",26,1,4,1,Winner
4,8305,DJ,Amanda,"$14,200",15,1,1,0,Second
5,8305,DJ,Devon,"$1,200",9,0,3,0,Third
6,8305,FJ,Brayden,"$3,999",26,1,4,1,Winner
7,8305,FJ,Amanda,"$3,200",15,1,1,0,Second
8,8305,FJ,Devon,"$1,200",9,0,3,0,Third


In [282]:
categories = soup.findAll('td', attrs = {'class': 'category_name'})

In [283]:
categories = np.array([x.text for x in categories])
categories

array(['STATES BY COUNTY', 'RUTH BADER GINSBURG',
       'DONATING THEIR WINNINGS', 'MOVING', 'YACHT ROCK', 'FINAL "RE"',
       'CHINESE HISTORY', '4-LETTER FILMS', 'LITERATURE', 'BIOLOGY',
       "MONEY MAKES THE WORLD GO 'ROUND", 'IN EXCELSIS DEO',
       'INNOVATIONS'], dtype='<U31')

In [284]:
rounds = []
[[rounds.append(item) for x in range(6)] for item in ['J', 'DJ']]
rounds.append('FJ')
rounds = np.array(rounds)
rounds

array(['J', 'J', 'J', 'J', 'J', 'J', 'DJ', 'DJ', 'DJ', 'DJ', 'DJ', 'DJ',
       'FJ'], dtype='<U2')

In [285]:
catnums = np.array(['1', '2', '3', '4', '5', '6', '1', '2', '3', '4', '5', '6', '1'])
catnums

array(['1', '2', '3', '4', '5', '6', '1', '2', '3', '4', '5', '6', '1'],
      dtype='<U1')

In [286]:
categories = pd.DataFrame({'category': categories, 'round_code': rounds, 'cat_num': catnums})

In [287]:
categories['season'] = season
categories['show_num'] = show_num
categories['air_date'] = air_date
categories

,category,round_code,cat_num,season,show_num,air_date
0,STATES BY COUNTY,J,1,37,8305,2020-12-18
1,RUTH BADER GINSBURG,J,2,37,8305,2020-12-18
2,DONATING THEIR WINNINGS,J,3,37,8305,2020-12-18
3,MOVING,J,4,37,8305,2020-12-18
4,YACHT ROCK,J,5,37,8305,2020-12-18
5,"FINAL ""RE""",J,6,37,8305,2020-12-18
6,CHINESE HISTORY,DJ,1,37,8305,2020-12-18
7,4-LETTER FILMS,DJ,2,37,8305,2020-12-18
8,LITERATURE,DJ,3,37,8305,2020-12-18
9,BIOLOGY,DJ,4,37,8305,2020-12-18


In [288]:
#split the results by the "clue" class
clues = soup.find_all('td', attrs = {'class': 'clue'})
print(type(clues))
print(type(soup))

<class 'bs4.element.ResultSet'>
<class 'bs4.BeautifulSoup'>


In [289]:
all_rows = []
for clue in clues:
    #print(type(clue))
    new_row = []
    #Get our clue number, which tells us the round, category, and board position
    clue_id = clue.find('td', class_='clue_text')['id']
    #What round is this?
    round_code = clue_id.split('_')[1]
    cat_num = clue_id.split('_')[2] if not round_code == 'FJ' else 'checkme'
    clue_num = clue_id.split('_')[3] if not round_code == 'FJ' else 'checkme'
    if not round_code == 'FJ':
        #Get our clue value
        #DD does not have a clue_value, it has a clue_value_daily_double
        #For DD split out the dollar value and set the flag
        clue_value = clue.find('td', class_='clue_value_daily_double').text.split(' ')[1] if not clue.find('td', {'class':'clue_value'}) else clue.find('td', {'class':'clue_value'}).text
        daily_double = False if not clue.find('td', class_='clue_value_daily_double') else True
        clue_order = clue.find('td', class_='clue_order_number').text
        clue_text = clue.find('td', class_='clue_text').text
        corr_resp = BeautifulSoup(clue.find('div', onmouseover=True).get('onmouseover'))
        #print(corr_resp)
        #Check for clues with no correct responder, and set the flag
        corr_respr1 = '' if not corr_resp.find('td', class_='right') else corr_resp.find('td', class_='right').text
        corr_respr2 = ''
        corr_respr3 = ''
        triple_stumper = True if not corr_resp.find('td', class_='right') else False
        incorr_resprs = corr_resp.findAll('td', class_='wrong')
        #Okay, THIS is ugly
        incorr_respr1 = ''
        incorr_respr2 = ''
        incorr_respr3 = ''
        if len(incorr_resprs) >= 1:
            incorr_respr1 = '' if incorr_resprs[0].text == 'Triple Stumper' else incorr_resprs[0].text
        if len(incorr_resprs) >= 2:
            incorr_respr2 = '' if incorr_resprs[1].text == 'Triple Stumper' else incorr_resprs[1].text
        if len(incorr_resprs) == 3:
            incorr_respr3 = '' if incorr_resprs[2].text == 'Triple Stumper' else incorr_resprs[2].text
        corr_resp = corr_resp.find('em', class_='correct_response').text
        new_row = [round_code, cat_num, clue_num, clue_id, clue_value, daily_double, clue_order, clue_text, corr_resp, 
                   corr_respr1, corr_respr2, corr_respr3, triple_stumper, incorr_respr1, incorr_respr2, incorr_respr3]
        all_rows.append(new_row)


In [290]:
round_code = 'FJ'
cat_num = '1'
clue_num = '1'
clue_id = 'clue_FJ'
clue_value = ''
daily_double = False
clue_order = ''
clue_text = ''
corr_resp = ''
triple_stumper = True

final_round = soup.find('table', class_ = 'final_round').find_all('td')
#type(final_round)
#final_round - this will absolutely barf and so will a print; it gives a type error

for item in final_round:
    #this only works because it's the first item!
    stats = BeautifulSoup(item.find('div', onmouseover=True).get('onmouseover'))
    if stats:
        break
corr_resp = stats.find('em')
corr_resp = corr_resp.get_text()

#make a dictionary of responders
resprs = {}
rights = stats.findAll('td', class_='right')
for resp in range(3):
    respr_col = 'corr_respr' + str(resp+1)
    resprs[respr_col] = rights[resp].get_text() if len(rights) >= resp+1 else ''
    triple_stumper = False

wrongs = stats.findAll('td', class_='wrong')
for resp in range(3):
    respr_col = 'incorr_respr' + str(resp+1)
    resprs[respr_col] = wrongs[resp].get_text() if len(wrongs) >= resp+1 else ''

corr_respr1 = resprs['corr_respr1']
corr_respr2 = resprs['corr_respr2']
corr_respr3 = resprs['corr_respr3']
incorr_respr1 = resprs['incorr_respr1']
incorr_respr2 = resprs['incorr_respr2']
incorr_respr3 = resprs['incorr_respr3']

clue_text = soup.find('td', attrs = {'class': 'clue_text', 'id': 'clue_FJ'})
clue_text = clue_text.get_text()
#print(clue_text)

new_row = [round_code, cat_num, clue_num, clue_id, clue_value, daily_double, clue_order, clue_text, corr_resp, 
                   corr_respr1, corr_respr2, corr_respr3, triple_stumper, incorr_respr1, incorr_respr2, incorr_respr3]
all_rows.append(new_row)

In [295]:
this_show = pd.DataFrame(all_rows)
this_show.columns = ['round_code','cat_num','clue_num','clue_id', 'clue_value', 'daily_double', 'clue_order', 'clue_text', 'corr_resp', 
              'corr_respr1','corr_respr2','corr_respr3','triple_stumper', 'incorr_respr1', 'incorr_respr2', 'incorr_respr3']
this_show = this_show.merge(categories, how='left', left_on=['round_code', 'cat_num'], right_on=['round_code', 'cat_num'])
this_show

,round_code,cat_num,clue_num,clue_id,clue_value,daily_double,clue_order,clue_text,corr_resp,corr_respr1,corr_respr2,corr_respr3,triple_stumper,incorr_respr1,incorr_respr2,incorr_respr3,category,season,show_num,air_date
0,J,1,1,clue_J_1_1,$200,False,10,"Kern, Imperial, Lassen",California,Devon,,,False,Brayden,,,STATES BY COUNTY,37,8305,2020-12-18
1,J,2,1,clue_J_2_1,$200,False,23,"Always in style, Justice Ginsburg was famous f...",dissent collar,Amanda,,,False,,,,RUTH BADER GINSBURG,37,8305,2020-12-18
2,J,3,1,clue_J_3_1,$200,False,30,After winning a 2020 tennis tourney in Aucklan...,the Australian fires,Brayden,,,False,,,,DONATING THEIR WINNINGS,37,8305,2020-12-18
3,J,4,1,clue_J_4_1,$200,False,28,$2.00 a day was the price when this trailer re...,U-Haul,Brayden,,,False,,,,MOVING,37,8305,2020-12-18
4,J,5,1,clue_J_5_1,$200,False,4,"Mike Campbell wrote the music for ""Boys Of Sum...",Don Henley,Devon,,,False,,,,YACHT ROCK,37,8305,2020-12-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,DJ,3,5,clue_DJ_3_5,$2000,False,7,"This playwright's ""The Children's Hour"" was in...",(Lillian) Hellman,Brayden,,,False,,,,LITERATURE,37,8305,2020-12-18
57,DJ,4,5,clue_DJ_4_5,$2000,False,12,"During this 13-letter process, plants lose wat...",transpiration,Brayden,,,False,,,,BIOLOGY,37,8305,2020-12-18
58,DJ,5,5,clue_DJ_5_5,$2000,False,9,The only remaining currency in Europe still na...,Liechtenstein,Amanda,,,False,Brayden,Devon,,MONEY MAKES THE WORLD GO 'ROUND,37,8305,2020-12-18
59,DJ,6,5,clue_DJ_6_5,$2000,False,19,Term for a very old manuscript volume of the s...,a codex,Brayden,,,False,,,,IN EXCELSIS DEO,37,8305,2020-12-18


In [297]:
this_show.to_csv('data/j_clues.csv', index=False)

In [276]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    13 non-null     object
 1   round_code  13 non-null     object
 2   cat_num     13 non-null     int32 
 3   season      13 non-null     object
 4   show_num    13 non-null     object
 5   air_date    13 non-null     object
dtypes: int32(1), object(5)
memory usage: 700.0+ bytes


In [277]:
this_show.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   round_code      61 non-null     object
 1   cat_num         61 non-null     object
 2   clue_num        61 non-null     object
 3   clue_id         61 non-null     object
 4   clue_value      61 non-null     object
 5   daily_double    61 non-null     bool  
 6   clue_order      61 non-null     object
 7   clue_text       61 non-null     object
 8   corr_resp       61 non-null     object
 9   corr_respr1     61 non-null     object
 10  corr_respr2     61 non-null     object
 11  corr_respr3     61 non-null     object
 12  triple_stumper  61 non-null     bool  
 13  incorr_respr1   61 non-null     object
 14  incorr_respr2   61 non-null     object
 15  incorr_respr3   61 non-null     object
dtypes: bool(2), object(14)
memory usage: 6.9+ KB


# Scratchpad

answer = BeautifulSoup(a.find("div", onmouseover=True).get("onmouseover"), "lxml")
answer = answer.find("em", class_="correct_response").get_text()

In [ ]:
corr_resp = BeautifulSoup(clue.find('div', onmouseover=True).get('onmouseover'))
corr_respr = corr_resp.find('td', class_='right').text
corr_resp = corr_resp.find('em', class_='correct_response').get_text()
print(corr_resp, ', ', corr_respr)


In [ ]:
new_row = []
all_rows = []
new_row = [clue_number, clue_value, clue_order, clue_text, corr_resp, corr_respr]
all_rows.append([new_row])
print(new_row)
print(all_rows)

In [ ]:
div_attrs = clue.find('div').attrs
for attr in div_attrs:
    print(type(attr), attr)
#    for tag in attr:
#        print(tag[attr])


In [ ]:
page = BeautifulSoup('<foo bar="asdf" blah="123">xyz</foo>')
for attr, value in page.find('foo').attrs.items():
    print(attr, "=", value, ' ', type(value))

In [ ]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

In [ ]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

In [ ]:
img = tonight.find("img")
desc = img['title']
print(desc)

In [ ]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

In [ ]:
#https://www.dataquest.io/blog/web-scraping-tutorial-python/

In [ ]:
s = '<div class="question" id="get attrs" name="python" x="something">Hello World</div>'
soup1 = BeautifulSoup(s)

In [ ]:
attributes_dictionary = soup1.find('div').attrs
print(attributes_dictionary)
# prints: {'id': 'get attrs', 'x': 'something', 'class': ['question'], 'name': 'python'}

In [ ]:
print(attributes_dictionary['class'][0])
# prints: question

In [ ]:
print soup1.find('div').get_text()
# prints: Hello World

https://github.com/whymarrh/jeopardy-parser/blob/master/parser.py